In [1]:
import pandas as pd
import numpy as np
import dropbox as db

# SFV EM 2020 File Import

In [2]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import dbx_token
dbx = db.Dropbox(dbx_token)

filename = '/Users/matthiashugli/Dropbox/bucket/euro2020/raw-analyse-sfv.csv'

df = pd.DataFrame()
file = pd.read_csv(filename, header=3, delimiter=',', low_memory=False)
df = df.append(file)

ln: config.py: File exists


/usr/local/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
df.drop(df.columns[0], axis=1, inplace=True)
df.loc[len(df.index)] = df.iloc[1]

In [4]:

# Create and Clean Header
## Need to promote header afterwards
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

# 6 standards, 7 standards_no, 12 indirekter_freistoss, 20 nahdistanzzone, 27 should_catch, should_go, should_back, should_stay, 50 technik_falsch
typos = {'standarts': 'standards', 'unnamed:_8': 'standards_no', 'ndirekter_freistoss': 'indirekter_freistoss', 'nahdistanzzpone': 'nahdistanzzone', 
'wenn_falsch_cgbs': 'should_catch', 'unnamed:_29': 'should_go', 'unnamed:_30': 'should_back', 'unnamed:_31': 'should_stay', 'unnamed:_51': 'technik_falsch', '': '',}

df.rename(columns=typos, inplace=True)
df.drop([0], inplace=True)

In [5]:
col_event_infos = df.columns[0:6].tolist()
col_type_standard = df.columns[8:14].tolist()
col_spielsituation = df.columns[14:21].tolist()
col_verhalten = df.columns[21:31].tolist()
col_technik = df.columns[31:51].tolist()
col_tore = df.columns[51:57].tolist()

print(df.columns[6])

standards


In [6]:
df.standards_no[df[df.columns[6]].isna() & df[df.columns[7]].isna() & df[df.columns[8]].isna() & df[df.columns[9]].isna() & df[df.columns[10]].isna() & df[df.columns[11]].isna() & df[df.columns[12]].isna() & df[df.columns[13]].isna()] = '1'

df.loc[df.standards == '1', 'ist_standard'] = 'ja'
df.loc[df.standards_no == '1', 'ist_standard'] = 'nein'

df = df.drop(columns=['standards', 'standards_no'])
standard_column = df.pop('ist_standard')
df.insert(6, 'ist_standard', standard_column)

### Cleansing Verhalten
df.loc[df.richtig == '1', 'korrektes_verhalten'] = 'ja'
df.loc[df.falsch == '1', 'korrektes_verhalten'] = 'nein'

df = df.drop(columns=['richtig', 'falsch'])
verhalten_column = df.pop('korrektes_verhalten')
df.insert(7, 'korrektes_verhalten', verhalten_column)

### Cleansing Technik
df.loc[df.technik_falsch == '1', 'korrekte_technik'] = 'ja'
df.loc[df.technik_korrekt == '1', 'korrekte_technik'] = 'nein'

df = df.drop(columns=['technik_falsch', 'technik_korrekt'])
technik_column = df.pop('korrekte_technik')
df.insert(8, 'korrekte_technik', technik_column)

In [7]:
### Unpivot Augang der Situation (Tor, kein Tor etc.)
unpivot = pd.melt(df, id_vars=df.columns[:45].tolist(), value_vars=df.columns[48:].tolist(), var_name='outcome', value_name='value_outcome')
unpivot.dropna(subset=['value_outcome'], inplace=True)
unpivot.drop(columns='value_outcome', inplace=True)

outcome_column = unpivot.pop('outcome')
unpivot.insert(9, 'outcome', outcome_column)
unpivot.reset_index(inplace=True)

### Unpivot ausführende Technik
#unpivot = pd.melt(unpivot, id_vars=unpivot.columns[:31].tolist(), value_vars=unpivot.columns[31:].tolist(), var_name='technik', value_name='value_technik')
#unpivot.dropna(subset=['value_technik'], inplace=True)
#unpivot.drop(columns='value_technik', inplace=True)

#outcome_column = unpivot.pop('technik')
#unpivot.insert(10, 'technik', outcome_column)

print(unpivot.dtypes)

index                           int64
spielcode                      object
th_name                        object
team                           object
paarung                        object
minuten                        object
topszene                       object
ist_standard                   object
korrektes_verhalten            object
korrekte_technik               object
outcome                        object
seitlicher_freistoss           object
eckball                        object
penalty                        object
direkter_freistoss             object
indirekter_freistoss           object
einwurf                        object
ball_durch_den_16er            object
hoher_ball                     object
pass_in_die_tiefe              object
rückpass_vor_der_grundlinie    object
rote_zone                      object
distanzschuss                  object
nahdistanzzone                 object
catch                          object
go                             object
stay        

In [9]:
for index, row in unpivot.iterrows():
    for x in row.iloc[1, :]:
        print(x)

IndexingError: Too many indexers

In [129]:
#unpivot.iloc[:, 31:] = unpivot.iloc[:, 31:].replace(np.nan, 0)
#unpivot.iloc[:, 31:] = unpivot.iloc[:, 31:].replace('1', 1)
unpivot.iloc[:, 31:] = unpivot.iloc[:, 31:].astype(str)

#result = []
#for v in unpivot.iloc[:5, 31:]:
#  print(v)
#  print(unpivot[v].values == 1)
  #for x in unpivot[v].values:
      

#print(result)

#unpivot['te_returned'] = result
result = []
x = 32
for data in unpivot.iloc[:2, x:]:
  #if x < 38:
  columns = unpivot[data]
  print(data)
  print(columns.values)
  for v in columns.values:
    if v == '1':
      result.append(data)
    else:
      result.append('nan')
  x += 1


  
  
print(len(result))
#print(result)

#unpivot['te_returned'] = result
#print(unpivot.iloc[:20, 32:])
  
#print((unpivot == 1).iloc[:5, 31:])


ballfangen
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
lenken_ohne_hecht
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
hecht_stand
['nan' '1' 'nan' ... 'nan' '1' 'nan']
beinweg
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
hecht_1_schritt
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
hecht_side_step
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
hecht_laufend
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
block
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
kleine_kreuz
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
grosse_kreuz
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
ballangriff
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
ballangriff_kontrast
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
reaktion_fuss
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
reaktion_hand
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
hohe_ball_fangen
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
te_returned
['nan' 'nan' 'nan' ... 'nan' 'nan' 'nan']
58000


In [77]:
unpivot = pd.melt(unpivot, id_vars=df.columns[:30].tolist(), value_vars=df.columns[])

SyntaxError: invalid syntax (<ipython-input-77-fb82ac65d625>, line 1)

In [110]:
df.loc[df.standards == '1', 'ist_standards'] = 'ja'
df.loc[df.standards_no == '1', 'ist_standards'] = 'nein'

df.loc[df.richtig == '1', 'korrektes_verhalten'] = 'ja'
df.loc[df.falsch == '1', 'korrektes_verhalten'] = 'nein'

df.loc[df.technik_falsch == '1', 'korrekte_technik'] = 'ja'
df.loc[df.technik_korrekt == '1', 'korrekte_technik'] = 'nein'

print(df.head(5))

  spielcode            th_name    team  paarung minuten topszene standards  \
1        VR  TUR Ugurcan Cakir  Türkei  Italien    2'09      NaN       NaN   
2        VR  TUR Ugurcan Cakir  Türkei  Italien    3'50      NaN       NaN   
3        VR  TUR Ugurcan Cakir  Türkei  Italien    6'49      NaN       NaN   
4        VR  TUR Ugurcan Cakir  Türkei  Italien   13'26      NaN       NaN   
5        VR  TUR Ugurcan Cakir  Türkei  Italien   14'02      NaN       NaN   

  standards_no seitlicher_freistoss eckball  ... technik_falsch tor_erhalten  \
1            1                  NaN     NaN  ...            NaN          NaN   
2            1                  NaN     NaN  ...            NaN          NaN   
3            1                  NaN     NaN  ...            NaN          NaN   
4            1                  NaN     NaN  ...            NaN          NaN   
5            1                  NaN     NaN  ...            NaN          NaN   

  kein_tor_gehalten kein_tor_daneben kein_tor_gebl